In [1]:
import tensorflow as tf
from kekeras.modelsls import load_model, Model
from keras import backend

def extractgraph(model_path):
    session = tf.Session()
    backend.set_session(session)
    
    model = load_model(model_path)
    graph_def = session.graph.as_graphdef()
    